In [1]:
from qiskit import QuantumCircuit
from qiskit.providers.aer import AerSimulator

## Encoding an input

Default Quantum states are 0.

In [2]:
qc = QuantumCircuit(3,3)
qc.measure([0,1,2],[0,1,2])
qc.draw()

┌─┐      
q_0: ┤M├──────
     └╥┘┌─┐   
q_1: ─╫─┤M├───
      ║ └╥┘┌─┐
q_2: ─╫──╫─┤M├
      ║  ║ └╥┘
c: 3/═╩══╩══╩═
      0  1  2

In [3]:
sim = AerSimulator()

In [4]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'000': 1024}

Let's input 3 into 3-Qubit Quantum Circuit.

$$3 = 0\times 2^2 + 1\times 2^1 + 1\times 2^0$$

In [5]:
qc = QuantumCircuit(3,3)
qc.x([0,1])
qc.measure([0,1,2],[0,1,2])
qc.draw()

┌───┐┌─┐   
q_0: ┤ X ├┤M├───
     ├───┤└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └┬─┬┘ ║ └╥┘
q_2: ─┤M├──╫──╫─
      └╥┘  ║  ║ 
c: 3/══╩═══╩══╩═
       2   0  1

In [6]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'011': 1024}

### Custom Project #1

Let's input 12 into 4-Qubit Quantum Circuit.

In [7]:
qc = QuantumCircuit(4,4)
qc.x([2,3])
qc.measure([0,1,2,3],[0,1,2,3])
qc.draw()

┌─┐         
q_0: ─────┤M├─────────
          └╥┘┌─┐      
q_1: ──────╫─┤M├──────
     ┌───┐ ║ └╥┘┌─┐   
q_2: ┤ X ├─╫──╫─┤M├───
     ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ X ├─╫──╫──╫─┤M├
     └───┘ ║  ║  ║ └╥┘
c: 4/══════╩══╩══╩══╩═
           0  1  2  3

In [8]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'1100': 1024}

## Addition

The addition of one bits is represented by XOR gates

$$
0 + 0 = 00 \\
0 + 1 = 01 \\
1 + 0 = 01 \\
1 + 1 = 10
$$

The job of the XOR gate is done by the 'controlled-NOT gate' (CNOT) in Quantum computers.

![](https://learn.qiskit.org/content/intro/images/aoc/cnot_xor.svg)

In [9]:
qc = QuantumCircuit(2,2)
qc.x(0)
qc.cx(0,1) # CNOT (controlled by q_0 and targeting q_1)
qc.measure([0,1], [0,1])
display(qc.draw())

job = sim.run(qc)
result = job.result()
result.get_counts()

┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

{'11': 1024}

Now we do addition without overwrite - half adder

![](https://learn.qiskit.org/content/intro/images/aoc/xor.svg)

In [10]:
qc = QuantumCircuit(3,3)
qc.x(0)
qc.cx(0,2)
qc.cx(1,2)
qc.measure([0,1,2], [0,1,2])
display(qc.draw())

job = sim.run(qc)
result = job.result()
result.get_counts()

┌───┐          ┌─┐      
q_0: ┤ X ├──■───────┤M├──────
     └───┘  │       └╥┘┌─┐   
q_1: ───────┼────■───╫─┤M├───
          ┌─┴─┐┌─┴─┐ ║ └╥┘┌─┐
q_2: ─────┤ X ├┤ X ├─╫──╫─┤M├
          └───┘└───┘ ║  ║ └╥┘
c: 3/════════════════╩══╩══╩═
                     0  1  2

{'101': 1024}

Now we implement half adder for $1+1 = 10$.
First, we can think below circuit.

In [11]:
qc = QuantumCircuit(3,3)
qc.x(0)
qc.x(1)
qc.cx(0,2)
qc.cx(1,2)
qc.measure([0,1,2], [0,1,2])
display(qc.draw())

job = sim.run(qc)
result = job.result()
result.get_counts()

┌───┐          ┌─┐      
q_0: ┤ X ├──■───────┤M├──────
     ├───┤  │       └╥┘┌─┐   
q_1: ┤ X ├──┼────■───╫─┤M├───
     └───┘┌─┴─┐┌─┴─┐ ║ └╥┘┌─┐
q_2: ─────┤ X ├┤ X ├─╫──╫─┤M├
          └───┘└───┘ ║  ║ └╥┘
c: 3/════════════════╩══╩══╩═
                     0  1  2

{'011': 1024}

But the output is 0 not 1. Then how to obtain '10'?

![](https://learn.qiskit.org/content/intro/images/aoc/ha_circuit.svg)

The Toffoli gate is 'controlled-controlled-not' gate. It has 3-bit inputs and outputs; if the first two bits are both set to 1, it inverts the third bit, otherwise all bits stay the same.

In [12]:
qc = QuantumCircuit(4,4)
qc.x(0)
qc.x(1)
qc.cx(0,2)
qc.cx(1,2)
qc.ccx(0,1,3)
qc.measure([0,1,2,3], [0,1,2,3])
display(qc.draw())

job = sim.run(qc)
result = job.result()
result.get_counts()

┌───┐                  ┌─┐      
q_0: ┤ X ├──■─────────■─────┤M├──────
     ├───┤  │         │     └╥┘┌─┐   
q_1: ┤ X ├──┼────■────■──────╫─┤M├───
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐ ║ └╥┘   
q_2: ─────┤ X ├┤ X ├──┼──┤M├─╫──╫────
          └───┘└───┘┌─┴─┐└╥┘ ║  ║ ┌─┐
q_3: ───────────────┤ X ├─╫──╫──╫─┤M├
                    └───┘ ║  ║  ║ └╥┘
c: 4/═════════════════════╩══╩══╩══╩═
                          2  0  1  3

{'1011': 1024}

### Custom Project #2

Now, let's describe $2+1=3$ in Quantim circuit.

First, it is same as $10 + 01 = 011$

In [13]:
qc = QuantumCircuit(13,4)

qc.x(0) # 01
qc.x(3) # 10

# 2^0 addition
qc.cx(0,5)
qc.cx(2,5)

# 2^1 addition
qc.cx(1,6)
qc.cx(3,6)

# Toffoli for 2^0
qc.ccx(0,2,7)

# Toffoli for 2^1
qc.ccx(1,3,8)

# Addition for 2^1 again
qc.cx(6,9)
qc.cx(7,9)

# Toffoli for 2^1 again
qc.ccx(6,7,10)

# Addition for 2^2
qc.cx(8,11)
qc.cx(10,11)

# Toffoli for 2^2
qc.ccx(8,10,12)

# 2^3: 12
# 2^2: 11
# 2^1: 9
# 2^0: 5

qc.measure([5,9,11,12], [0,1,2,3])
display(qc.draw())

job = sim.run(qc)
result = job.result()
result.get_counts()

┌───┐                                                              
 q_0: ┤ X ├───────■───────────────────■──────────────────────────────────
      └───┘       │                   │                                  
 q_1: ───────■────┼──────────────■────┼──────────────────────────────────
             │    │              │    │                                  
 q_2: ───────┼────┼─────────■────┼────■──────────────────────────────────
      ┌───┐  │    │         │    │    │                                  
 q_3: ┤ X ├──┼────┼────■────┼────■────┼──────────────────────────────────
      └───┘  │    │    │    │    │    │                                  
 q_4: ───────┼────┼────┼────┼────┼────┼──────────────────────────────────
             │  ┌─┴─┐  │  ┌─┴─┐  │    │  ┌─┐                             
 q_5: ───────┼──┤ X ├──┼──┤ X ├──┼────┼──┤M├─────────────────────────────
           ┌─┴─┐└───┘┌─┴─┐└───┘  │    │  └╥┘                             
 q_6: ─────┤ X ├─────┤ X ├──■────┼────┼───╫────────■─────────────────────
           └───┘     └───┘  │    │  ┌─┴─┐ ║        │                     
 q_7: ──────────────────────┼────┼──┤ X ├─╫───■────■─────────────────────
                            │  ┌─┴─┐└───┘ ║   │    │                     
 q_8: ──────────────────────┼──┤ X ├──■───╫───┼────┼────────────■────────
                          ┌─┴─┐└───┘  │   ║ ┌─┴─┐  │  ┌─┐       │        
 q_9: ────────────────────┤ X ├───────┼───╫─┤ X ├──┼──┤M├───────┼────────
                          └───┘       │   ║ └───┘┌─┴─┐└╥┘       │        
q_10: ────────────────────────────────┼───╫──────┤ X ├─╫───■────■────────
                                    ┌─┴─┐ ║      └───┘ ║ ┌─┴─┐  │  ┌─┐   
q_11: ──────────────────────────────┤ X ├─╫────────────╫─┤ X ├──┼──┤M├───
                                    └───┘ ║            ║ └───┘┌─┴─┐└╥┘┌─┐
q_12: ────────────────────────────────────╫────────────╫──────┤ X ├─╫─┤M├
                                          ║            ║      └───┘ ║ └╥┘
 c: 4/════════════════════════════════════╩════════════╩════════════╩══╩═
                                          0            1            2  3

{'0011': 1024}

Above circuit may work with $2+3=5 ~\Rightarrow~ 10+11=101$

In [14]:
qc = QuantumCircuit(13,4)

qc.x(1) # 10
qc.x(2) # 11
qc.x(3) # 11

# 2^0 addition
qc.cx(0,5)
qc.cx(2,5)

# 2^1 addition
qc.cx(1,6)
qc.cx(3,6)

# Toffoli for 2^0
qc.ccx(0,2,7)

# Toffoli for 2^1
qc.ccx(1,3,8)

# Addition for 2^1 again
qc.cx(6,9)
qc.cx(7,9)

# Toffoli for 2^1 again
qc.ccx(6,7,10)

# Addition for 2^2
qc.cx(8,11)
qc.cx(10,11)

# Toffoli for 2^2
qc.ccx(8,10,12)

# 2^3: 12
# 2^2: 11
# 2^1: 9
# 2^0: 5

qc.measure([5,9,11,12], [0,1,2,3])
display(qc.draw())

job = sim.run(qc)
result = job.result()
result.get_counts()

»
 q_0: ───────■───────────────────■───────────────────────────────────────────»
      ┌───┐  │                   │                                           »
 q_1: ┤ X ├──┼─────────■─────────┼───────■───────────────────────────────────»
      ├───┤  │         │         │       │                                   »
 q_2: ┤ X ├──┼────■────┼─────────■───────┼───────────────────────────────────»
      ├───┤  │    │    │         │       │                                   »
 q_3: ┤ X ├──┼────┼────┼────■────┼───────■───────────────────────────────────»
      └───┘  │    │    │    │    │       │                                   »
 q_4: ───────┼────┼────┼────┼────┼───────┼───────────────────────────────────»
           ┌─┴─┐┌─┴─┐  │    │    │  ┌─┐  │                                   »
 q_5: ─────┤ X ├┤ X ├──┼────┼────┼──┤M├──┼───────────────────────────────────»
           └───┘└───┘┌─┴─┐┌─┴─┐  │  └╥┘  │                                   »
 q_6: ───────────────┤ X ├┤ X ├──┼───╫───┼────■──────────────■───────────────»
                     └───┘└───┘┌─┴─┐ ║   │    │              │               »
 q_7: ─────────────────────────┤ X ├─╫───┼────┼────■─────────■───────────────»
                               └───┘ ║ ┌─┴─┐  │    │         │               »
 q_8: ───────────────────────────────╫─┤ X ├──┼────┼────■────┼────────────■──»
                                     ║ └───┘┌─┴─┐┌─┴─┐  │    │  ┌─┐       │  »
 q_9: ───────────────────────────────╫──────┤ X ├┤ X ├──┼────┼──┤M├───────┼──»
                                     ║      └───┘└───┘  │  ┌─┴─┐└╥┘       │  »
q_10: ───────────────────────────────╫──────────────────┼──┤ X ├─╫───■────■──»
                                     ║                ┌─┴─┐└───┘ ║ ┌─┴─┐  │  »
q_11: ───────────────────────────────╫────────────────┤ X ├──────╫─┤ X ├──┼──»
                                     ║                └───┘      ║ └───┘┌─┴─┐»
q_12: ───────────────────────────────╫───────────────────────────╫──────┤ X ├»
                                     ║                           ║      └───┘»
 c: 4/═══════════════════════════════╩═══════════════════════════╩═══════════»
                                     0                           1           »
«            
« q_0: ──────
«            
« q_1: ──────
«            
« q_2: ──────
«            
« q_3: ──────
«            
« q_4: ──────
«            
« q_5: ──────
«            
« q_6: ──────
«            
« q_7: ──────
«            
« q_8: ──────
«            
« q_9: ──────
«            
«q_10: ──────
«      ┌─┐   
«q_11: ┤M├───
«      └╥┘┌─┐
«q_12: ─╫─┤M├
«       ║ └╥┘
« c: 4/═╩══╩═
«       2  3

{'0101': 1024}